Let's understand the dataset layout, to help us create a good database design.

In [233]:
#import libraries
import pandas as pd
import os

In [167]:
#Import the CSV. Encoding as per previous step.
df = pd.read_csv('Superstore.csv', sep=',', encoding_errors='ignore')

In [168]:
# Look at columns, size, nulls, datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [169]:
#First look at contents
df.head(5)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [170]:
#We need to fix OrderDate and ship date to be YYYY/MM/DD (can be ordered easily even when seen as a string). 
df[['Order Date','Ship Date']] = df[['Order Date','Ship Date']].apply(pd.to_datetime)

In [171]:
df.head(5)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [172]:
# Two new columns required for more in depth analysis. 
# Supplier price (what VS Group buy it for) and Catalogue price (what VS Group offer it for)

df["Catalogue price"] = (df.Sales/df.Quantity)/(1-df.Discount)

df["Supplier price"] = (df.Sales-df.Profit)/df.Quantity

df.head(5)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Catalogue price,Supplier price
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,130.98,110.0232
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,243.98,170.7860
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,7.31,3.8743
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,348.21,268.1217
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,13.98,9.9258


In [173]:
#Let's get these column names into something more SQLitey, we will be thankful later. 
rename_dict = {
        'Row ID': 'RowID',
        'Order ID': 'OrderID',
        'Order Date': 'OrderDate',
        'Ship Date': 'ShipDate',
        'Ship Mode': 'ShipMode',
        'Customer ID': 'CustomerID',
        'Customer Name': 'CustomerName',
        'Postal Code': 'PostalCode',
        'Product ID': 'ProductID',
        'Sub-Category': 'SubCategory',
        'Product Name': 'ProductName',
        'Catalogue price': 'CataloguePrice',
        'Supplier price': 'SupplierPrice'
        }
 
# call rename () method
df.rename(columns=rename_dict,
          inplace=True)

In [174]:
df.columns

Index(['RowID', 'OrderID', 'OrderDate', 'ShipDate', 'ShipMode', 'CustomerID',
       'CustomerName', 'Segment', 'Country', 'City', 'State', 'PostalCode',
       'Region', 'ProductID', 'Category', 'SubCategory', 'ProductName',
       'Sales', 'Quantity', 'Discount', 'Profit', 'CataloguePrice',
       'SupplierPrice'],
      dtype='object')

In [175]:
#Sales, Profit and Supplier Price are funky as heck. They need to be rounded to the nearest cent. 
#All prices will also need to be in CENTS to avoid floating point errors in SQLite.

cols = ["Sales", "Profit", "SupplierPrice", "CataloguePrice"]

df.loc[:, cols] = (df[cols]*100).astype(int)

Now let's attempt to break up the dataframe into component tables that will later be relational tables in our RDBMS.
This may take some trial and error offline with ERD and sketches.
We will ascribe PK, FK, datatypes and any other constraints later.

In [176]:
#Reminder of contents..
df.columns

Index(['RowID', 'OrderID', 'OrderDate', 'ShipDate', 'ShipMode', 'CustomerID',
       'CustomerName', 'Segment', 'Country', 'City', 'State', 'PostalCode',
       'Region', 'ProductID', 'Category', 'SubCategory', 'ProductName',
       'Sales', 'Quantity', 'Discount', 'Profit', 'CataloguePrice',
       'SupplierPrice'],
      dtype='object')

In [177]:
#Just trying some breakdowns to get a feel for the dataset.
#We can see that "Profit" is really just based on the catalogue price, we can still make money on "negative" sales.

#dfs.groupby(['State'])[["Sales", "Profit", "CataloguePrice", "SupplierPrice"]].sum().sort_values(by=['Sales'], ascending=False)

In [178]:
# frst pass

Customer = df[["CustomerID", "CustomerName", "Segment"]]
Address = df[["CustomerID", "Country", "State", "City", "PostalCode"]] # Address ID will be required to connect this to orders. Region will be removed.
Product = df[["ProductID", "ProductName", "CataloguePrice", "SupplierPrice"]] #Needs Cat ID!
Category = df[["Category", "SubCategory"]] #Category ID will be needed and linked back to Products
Order = df[["OrderID", "CustomerID", "OrderDate", "ShipDate", "ShipMode"]] #Needs Address ID!
OrderDetail = df[["OrderID", "ProductID", "Quantity", "Sales", "Discount", "Profit"]] #Sales table did not survive 3f

In [179]:
Customer.isnull().sum()

CustomerID      0
CustomerName    0
Segment         0
dtype: int64

In [180]:
Customer.head(3)

,CustomerID,CustomerName,Segment
0,CG-12520,Claire Gute,Consumer
1,CG-12520,Claire Gute,Consumer
2,DV-13045,Darrin Van Huff,Corporate


In [181]:
Customer["CustomerID"].unique().size

793

In [182]:
Customer["CustomerName"].unique().size

793

In [183]:
Customer = Customer.drop_duplicates(ignore_index=True)

In [184]:
Customer

,CustomerID,CustomerName,Segment
0,CG-12520,Claire Gute,Consumer
1,DV-13045,Darrin Van Huff,Corporate
2,SO-20335,Sean O'Donnell,Consumer
3,BH-11710,Brosina Hoffman,Consumer
4,AA-10480,Andrew Allen,Consumer
...,...,...,...
788,CJ-11875,Carl Jackson,Corporate
789,RS-19870,Roy Skaria,Home Office
790,SC-20845,Sung Chung,Consumer
791,RE-19405,Ricardo Emerson,Consumer


In [185]:
Address.isnull().sum()

CustomerID    0
Country       0
State         0
City          0
PostalCode    0
dtype: int64

In [186]:
Address.head(3)

,CustomerID,Country,State,City,PostalCode
0,CG-12520,United States,Kentucky,Henderson,42420
1,CG-12520,United States,Kentucky,Henderson,42420
2,DV-13045,United States,California,Los Angeles,90036


In [187]:
#The goal here is to see if customers have multiple addresses.
Address.groupby(['CustomerID', 'PostalCode']).ngroups

4910

In [188]:
Address['CustomerID'].unique().size

793

In [189]:
Address=Address.drop_duplicates()

In [190]:
duplicate = Address[Address.duplicated('CustomerID')]

In [191]:
duplicate.sort_values(by='CustomerID', ascending=False)

,CustomerID,Country,State,City,PostalCode
3814,ZD-21925,United States,Florida,Jacksonville,32216
3040,ZD-21925,United States,Tennessee,Chattanooga,37421
5897,ZD-21925,United States,Indiana,Richmond,47374
8341,ZD-21925,United States,California,Los Angeles,90036
8923,ZC-21910,United States,North Carolina,Hickory,28601
...,...,...,...,...,...
807,AA-10375,United States,Nebraska,Omaha,68104
1299,AA-10315,United States,California,San Francisco,94109
2229,AA-10315,United States,California,San Francisco,94122
7468,AA-10315,United States,New York,New York City,10011


In [192]:
#We need to make a new column to use as a primary key for Addresses. That, or use a composite primary.. 
# Good discussion here https://stackoverflow.com/questions/14588304/composite-primary-key-vs-additional-id-column
#So we will use these as a reference to increase the number and we will put UNIQUE constraints on the composite, but use the ID as primary key.
Address['AddressID'] = range(1, len(Address) + 1)

In [193]:
Address['AddressID'] = "A" + Address['AddressID'].astype(str).str.zfill(6)

In [194]:
Address

,CustomerID,Country,State,City,PostalCode,AddressID
0,CG-12520,United States,Kentucky,Henderson,42420,A000001
2,DV-13045,United States,California,Los Angeles,90036,A000002
3,SO-20335,United States,Florida,Fort Lauderdale,33311,A000003
5,BH-11710,United States,California,Los Angeles,90032,A000004
12,AA-10480,United States,North Carolina,Concord,28027,A000005
...,...,...,...,...,...,...
9986,ML-17410,United States,California,Los Angeles,90008,A004906
9987,RA-19885,United States,Georgia,Athens,30605,A004907
9989,TB-21400,United States,Florida,Miami,33180,A004908
9990,DB-13060,United States,California,Costa Mesa,92627,A004909


In [195]:
Address.reset_index(drop=True)

,CustomerID,Country,State,City,PostalCode,AddressID
0,CG-12520,United States,Kentucky,Henderson,42420,A000001
1,DV-13045,United States,California,Los Angeles,90036,A000002
2,SO-20335,United States,Florida,Fort Lauderdale,33311,A000003
3,BH-11710,United States,California,Los Angeles,90032,A000004
4,AA-10480,United States,North Carolina,Concord,28027,A000005
...,...,...,...,...,...,...
4905,ML-17410,United States,California,Los Angeles,90008,A004906
4906,RA-19885,United States,Georgia,Athens,30605,A004907
4907,TB-21400,United States,Florida,Miami,33180,A004908
4908,DB-13060,United States,California,Costa Mesa,92627,A004909


In [196]:
#Here I want to see if the prices remain consistent throughout the dataset. 
#Futureprooding would definitely require us to be able to change catalogue or Supplier price going forward.
price_test = df[["ProductID","CataloguePrice","SupplierPrice"]]

In [197]:
price_test.sort_values('ProductID')

,ProductID,CataloguePrice,SupplierPrice
3512,FUR-BO-10000112,13097.0,10478.0
5494,FUR-BO-10000330,12098.0,10404.0
6401,FUR-BO-10000330,12098.0,10404.0
1760,FUR-BO-10000330,12098.0,10404.0
1594,FUR-BO-10000362,17098.0,13165.0
...,...,...,...
6635,TEC-PH-10004977,19598.0,13915.0
5064,TEC-PH-10004977,19599.0,13915.0
9727,TEC-PH-10004977,19599.0,13915.0
7477,TEC-PH-10004977,19599.0,13915.0


In [198]:
price_test = price_test.drop_duplicates(ignore_index=True)

In [199]:
#Many of these duplicates were not being caught due to rounding errors at the calculation stage despite precautions. 
price_test.sort_values('ProductID', ignore_index=True)

,ProductID,CataloguePrice,SupplierPrice
0,FUR-BO-10000112,13097.0,10478.0
1,FUR-BO-10000330,12098.0,10404.0
2,FUR-BO-10000362,17098.0,13165.0
3,FUR-BO-10000468,4857.0,4080.0
4,FUR-BO-10000468,4858.0,4080.0
...,...,...,...
2734,TEC-PH-10004922,6699.0,5024.0
2735,TEC-PH-10004924,739.0,539.0
2736,TEC-PH-10004959,10049.0,7536.0
2737,TEC-PH-10004977,19599.0,13915.0


In [200]:
Category

,Category,SubCategory
0,Furniture,Bookcases
1,Furniture,Chairs
2,Office Supplies,Labels
3,Furniture,Tables
4,Office Supplies,Storage
...,...,...
9989,Furniture,Furnishings
9990,Furniture,Furnishings
9991,Technology,Phones
9992,Office Supplies,Paper


In [201]:
#Ok, same approach as with addresses.
Category.groupby(['Category', 'SubCategory']).ngroups

17

In [202]:
Category=Category.drop_duplicates(ignore_index=True)

In [203]:
Category = Category.sort_values(by='Category')

In [204]:
Category['CategoryID'] = range(1, len(Category) + 1)

In [205]:
#Allowing up to 9999 categories with current format. Could also just use a different letter later but this seems neater. 
#An E-Commerce website could expand its number of categories pretty easily. 
Category['CategoryID'] = "C" + Category['CategoryID'].astype(str).str.zfill(4)

In [206]:
Category

,Category,SubCategory,CategoryID
0,Furniture,Bookcases,C0001
1,Furniture,Chairs,C0002
3,Furniture,Tables,C0003
5,Furniture,Furnishings,C0004
14,Office Supplies,Supplies,C0005
13,Office Supplies,Fasteners,C0006
12,Office Supplies,Envelopes,C0007
10,Office Supplies,Paper,C0008
9,Office Supplies,Appliances,C0009
8,Office Supplies,Binders,C0010


In [207]:
Category.columns

Index(['Category', 'SubCategory', 'CategoryID'], dtype='object')

In [208]:
df2 = Category.merge(df, left_on=['Category', 'SubCategory'], right_on=['Category', 'SubCategory'], how='inner').sort_values(by="RowID", ignore_index=True)

In [209]:
df2

,Category,SubCategory,CategoryID,RowID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,...,PostalCode,Region,ProductID,ProductName,Sales,Quantity,Discount,Profit,CataloguePrice,SupplierPrice
0,Furniture,Bookcases,C0001,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,...,42420,South,FUR-BO-10001798,Bush Somerset Collection Bookcase,26195.0,2,0.00,4191.0,13097.0,11002.0
1,Furniture,Chairs,C0002,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,...,42420,South,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",73194.0,3,0.00,21958.0,24398.0,17078.0
2,Office Supplies,Labels,C0013,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,...,90036,West,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,1462.0,2,0.00,687.0,731.0,387.0
3,Furniture,Tables,C0003,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,...,33311,South,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,95757.0,5,0.45,-38303.0,34821.0,26812.0
4,Office Supplies,Storage,C0012,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,...,33311,South,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,2236.0,2,0.20,251.0,1397.0,992.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,Furniture,Furnishings,C0004,9990,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,...,33180,South,FUR-FU-10001889,Ultra Door Pull Handle,2524.0,3,0.20,410.0,1052.0,704.0
9990,Furniture,Furnishings,C0004,9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,...,92627,West,FUR-FU-10000747,Tenex B1-RE Series Chair Mats for Low Pile Car...,9196.0,2,0.00,1563.0,4598.0,3816.0
9991,Technology,Phones,C0014,9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,...,92627,West,TEC-PH-10003645,Aastra 57i VoIP phone,25857.0,2,0.20,1939.0,16161.0,11959.0
9992,Office Supplies,Paper,C0008,9993,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,...,92627,West,OFF-PA-10004041,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",2960.0,4,0.00,1332.0,740.0,407.0


In [211]:
Product = df2[["ProductID", "ProductName", "CataloguePrice", "SupplierPrice", "CategoryID"]]

In [212]:
Product.head(3)

,ProductID,ProductName,CataloguePrice,SupplierPrice,CategoryID
0,FUR-BO-10001798,Bush Somerset Collection Bookcase,13097.0,11002.0,C0001
1,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",24398.0,17078.0,C0002
2,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,731.0,387.0,C0013


In [213]:
Product = Product.drop_duplicates(subset=["ProductID", "ProductName"])

In [214]:
Product

,ProductID,ProductName,CataloguePrice,SupplierPrice,CategoryID
0,FUR-BO-10001798,Bush Somerset Collection Bookcase,13097.0,11002.0,C0001
1,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",24398.0,17078.0,C0002
2,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,731.0,387.0,C0013
3,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,34821.0,26812.0,C0003
4,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,1397.0,992.0,C0012
...,...,...,...,...,...
9521,TEC-PH-10002817,RCA ViSYS 25425RE1 Corded phone,13499.0,9584.0,C0014
9562,TEC-MA-10003589,Cisco 8961 IP Phone Charcoal,24992.0,12996.0,C0016
9604,OFF-AP-10003099,"Eureka Hand Vacuum, Bagless",4943.0,3558.0,C0009
9673,TEC-PH-10002645,LG G2,49999.0,37499.0,C0014


In [215]:
Product[Product.groupby('ProductID')['ProductID'].transform('size') > 1].sort_values(by=['ProductID'])

,ProductID,ProductName,CataloguePrice,SupplierPrice,CategoryID
2471,FUR-BO-10002213,"Sauder Forest Hills Library, Woodland Oak Finish",14097.0,12970.0,C0001
2115,FUR-BO-10002213,DMI Eclipse Executive Suite Bookcases,50098.0,40579.0,C0001
66,FUR-CH-10001146,"Global Value Mid-Back Manager's Chair, Gray",6089.0,4566.0,C0002
128,FUR-CH-10001146,"Global Task Chair, Black",5089.0,4529.0,C0002
1459,FUR-FU-10001473,DAX Wood Document Frame,1373.0,878.0,C0004
...,...,...,...,...,...
1219,TEC-PH-10002200,Samsung Galaxy Note 2,45999.0,33119.0,C0014
2596,TEC-PH-10002310,Plantronics Calisto P620-M USB Wireless Speake...,19599.0,14307.0,C0014
1378,TEC-PH-10002310,Panasonic KX T7731-B Digital phone,9998.0,7299.0,C0014
922,TEC-PH-10004531,OtterBox Commuter Series Case - iPhone 5 & 5s,2199.0,1143.0,C0014


In [216]:
Product[Product.groupby('ProductID')['ProductID'].transform('size') > 2].sort_values(by=['ProductID'])

,ProductID,ProductName,CataloguePrice,SupplierPrice,CategoryID


In [144]:
#So we have a bunch of products who share IDs and all do seem to belong in the correct category.
#So we must manage them with as little fuss as possible. If stock numbers were known then we would change ones with lower num. items in stock (less work)
#But we don't know this so let's modify every second one. As we can see above, never more than one duplicate.

In [217]:
#Let's find the biggest value in the column and work from there. 
#I imagine all products are added iteratively i.e. a new product is assigned the next available number regardless of category and subcategory.
Product['ProductID'].str[-4:].max()

'4999'

In [218]:
# Sort the DataFrame based on 'ProductID' to make sure the IDs are in order
Product.sort_values('ProductID', inplace=True)

# Calculate the current_count based on the maximum value of the last four digits
current_count = int(Product['ProductID'].str[-4:].max())

# Iterate through the DataFrame and update the duplicated ProductIDs
for index, row in Product.iterrows():
    product_id = row['ProductID']
    
    # Check if the current ProductID is a duplicate
    if Product[Product['ProductID'] == product_id].shape[0] > 1:
        # Increment the count and update the ProductID
        current_count += 1
        new_product_id = f"{product_id[:-4]}{current_count:04d}"
        Product.loc[index, 'ProductID'] = new_product_id

# Reset the index
Product.reset_index(drop=True, inplace=True)


In [219]:
Product

,ProductID,ProductName,CataloguePrice,SupplierPrice,CategoryID
0,FUR-BO-10000112,"Bush Birmingham Collection Bookcase, Dark Cherry",13097.0,10478.0,C0001
1,FUR-BO-10000330,"Sauder Camden County Barrister Bookcase, Plank...",12098.0,10404.0,C0001
2,FUR-BO-10000362,Sauder Inglewood Library Bookcases,17098.0,13165.0,C0001
3,FUR-BO-10000468,O'Sullivan 2-Shelf Heavy-Duty Bookcases,4858.0,4080.0,C0001
4,FUR-BO-10000711,"Hon Metal Bookcases, Gray",7098.0,5181.0,C0001
...,...,...,...,...,...
1889,TEC-PH-10004912,Cisco SPA112 2 Port Phone Adapter,5495.0,3901.0,C0014
1890,TEC-PH-10004922,RCA Visys Integrated PBX 8-Line Router,6699.0,5024.0,C0014
1891,TEC-PH-10004924,"SKILCRAFT Telephone Shoulder Rest, 2"" x 6.5"" x...",739.0,539.0,C0014
1892,TEC-PH-10004959,Classic Ivory AntiqueTelephoneZL1810,10049.0,7536.0,C0014


In [220]:
Product['ProductID'].str[-4:].max()

'5031'

In [221]:
Address.columns

Index(['CustomerID', 'Country', 'State', 'City', 'PostalCode', 'AddressID'], dtype='object')

In [222]:
df3 = Address.merge(df2, left_on=['CustomerID', 'Country', 'State', 'City', 'PostalCode'], 
                     right_on=['CustomerID', 'Country', 'State', 'City', 'PostalCode'], 
                     how='inner').sort_values(by="RowID", ignore_index=True)

In [223]:
df3.columns

Index(['CustomerID', 'Country', 'State', 'City', 'PostalCode', 'AddressID',
       'Category', 'SubCategory', 'CategoryID', 'RowID', 'OrderID',
       'OrderDate', 'ShipDate', 'ShipMode', 'CustomerName', 'Segment',
       'Region', 'ProductID', 'ProductName', 'Sales', 'Quantity', 'Discount',
       'Profit', 'CataloguePrice', 'SupplierPrice'],
      dtype='object')

In [224]:
Order = df3[["OrderID", "CustomerID", "OrderDate", "AddressID"]]

In [225]:
Order

,OrderID,CustomerID,OrderDate,AddressID
0,CA-2016-152156,CG-12520,2016-11-08,A000001
1,CA-2016-152156,CG-12520,2016-11-08,A000001
2,CA-2016-138688,DV-13045,2016-06-12,A000002
3,US-2015-108966,SO-20335,2015-10-11,A000003
4,US-2015-108966,SO-20335,2015-10-11,A000003
...,...,...,...,...
9989,CA-2014-110422,TB-21400,2014-01-21,A004908
9990,CA-2017-121258,DB-13060,2017-02-26,A004909
9991,CA-2017-121258,DB-13060,2017-02-26,A004909
9992,CA-2017-121258,DB-13060,2017-02-26,A004909


In [226]:
Order = Order.drop_duplicates(subset="OrderID", ignore_index=True)

In [227]:
Order = Order.sort_values('OrderDate', ascending=True, ignore_index=True)

In [228]:
Order

,OrderID,CustomerID,OrderDate,AddressID
0,CA-2014-103800,DP-13000,2014-01-03,A003887
1,CA-2014-112326,PO-19195,2014-01-04,A000343
2,CA-2014-141817,MB-18085,2014-01-05,A000854
3,CA-2014-106054,JO-15145,2014-01-06,A003509
4,CA-2014-130813,LS-17230,2014-01-06,A002614
...,...,...,...,...
5004,CA-2017-130631,BS-11755,2017-12-29,A002686
5005,CA-2017-143259,PO-18865,2017-12-30,A000432
5006,CA-2017-115427,EB-13975,2017-12-30,A000610
5007,CA-2017-126221,CC-12430,2017-12-30,A000300


In [229]:
OrderDetail = OrderDetail.drop_duplicates(subset=["OrderID", "ProductID"], ignore_index=True)

In [230]:
OrderDetail

,OrderID,ProductID,Quantity,Sales,Discount,Profit
0,CA-2016-152156,FUR-BO-10001798,2,26195.0,0.00,4191.0
1,CA-2016-152156,FUR-CH-10000454,3,73194.0,0.00,21958.0
2,CA-2016-138688,OFF-LA-10000240,2,1462.0,0.00,687.0
3,US-2015-108966,FUR-TA-10000577,5,95757.0,0.45,-38303.0
4,US-2015-108966,OFF-ST-10000760,2,2236.0,0.20,251.0
...,...,...,...,...,...,...
9981,CA-2014-110422,FUR-FU-10001889,3,2524.0,0.20,410.0
9982,CA-2017-121258,FUR-FU-10000747,2,9196.0,0.00,1563.0
9983,CA-2017-121258,TEC-PH-10003645,2,25857.0,0.20,1939.0
9984,CA-2017-121258,OFF-PA-10004041,4,2960.0,0.00,1332.0


In [231]:
df3.head(3)

,CustomerID,Country,State,City,PostalCode,AddressID,Category,SubCategory,CategoryID,RowID,...,Segment,Region,ProductID,ProductName,Sales,Quantity,Discount,Profit,CataloguePrice,SupplierPrice
0,CG-12520,United States,Kentucky,Henderson,42420,A000001,Furniture,Bookcases,C0001,1,...,Consumer,South,FUR-BO-10001798,Bush Somerset Collection Bookcase,26195.0,2,0.0,4191.0,13097.0,11002.0
1,CG-12520,United States,Kentucky,Henderson,42420,A000001,Furniture,Chairs,C0002,2,...,Consumer,South,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",73194.0,3,0.0,21958.0,24398.0,17078.0
2,DV-13045,United States,California,Los Angeles,90036,A000002,Office Supplies,Labels,C0013,3,...,Corporate,West,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,1462.0,2,0.0,687.0,731.0,387.0


In [232]:
#Now I want to output CSVs just for flexibility and to be safe.
list_of_tables = ['Customer', 'Address', 'Product', 'Category', 'Order', 'OrderDetail']
dataframes = [Customer, Address, Product, Category, Order, OrderDetail]

In [234]:
#Here we take the list of tables and dataframes we want to turn into .csvs and save them to the directory we defined in the beginning
for i in range (len(list_of_tables)):
    x = dataframes[i]
    y = list_of_tables[i]
    x.to_csv(f"{'/Users/penguin/Desktop/CodeAcademy/Ecommerce/'}/{y}.csv", index=False)

In [236]:
#For safety I am also saving the full, final dataframe one folder up in the same directory.  
up_path = os.path.dirname(os.path.dirname('/Users/penguin/Desktop/CodeAcademy/Ecommerce/'))
df3.to_csv(f'{up_path}/db.csv', index=False)

In [237]:
#Lazy way to see longest value in each column. Useful later when creating db table constraints.
for col in df3:
    print(f'Longest val in {col} is {df3[col].astype(str).map(len).max()}, min is {df3[col].astype(str).map(len).min()}')

Longest val in CustomerID is 8, min is 8
Longest val in Country is 13, min is 13
Longest val in State is 20, min is 4
Longest val in City is 17, min is 4
Longest val in PostalCode is 5, min is 4
Longest val in AddressID is 7, min is 7
Longest val in Category is 15, min is 9
Longest val in SubCategory is 11, min is 3
Longest val in CategoryID is 5, min is 5
Longest val in RowID is 4, min is 1
Longest val in OrderID is 14, min is 14
Longest val in OrderDate is 10, min is 10
Longest val in ShipDate is 10, min is 10
Longest val in ShipMode is 14, min is 8
Longest val in CustomerName is 22, min is 7
Longest val in Segment is 11, min is 8
Longest val in Region is 7, min is 4
Longest val in ProductID is 15, min is 15
Longest val in ProductName is 127, min is 5
Longest val in Sales is 9, min is 4
Longest val in Quantity is 2, min is 1
Longest val in Discount is 4, min is 3
Longest val in Profit is 9, min is 3
Longest val in CataloguePrice is 8, min is 4
Longest val in SupplierPrice is 8, min i

Over to dbcreation.ipynb for the database creation! 